In [5]:
!pip install transformers.AdamW -q

ERROR: Could not find a version that satisfies the requirement transformers.AdamW (from versions: none)
ERROR: No matching distribution found for transformers.AdamW


In [ ]:
#@title 1. Instalasi dan Impor Pustaka
# ------------------------------------------------------------------
# Menginstal pustaka yang dibutuhkan.
# ------------------------------------------------------------------
import json
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer, util
from torch.optim import AdamW
import numpy as np

print("✅ Pustaka berhasil diinstal dan diimpor.")

✅ Pustaka berhasil diinstal dan diimpor.


In [ ]:
#@title 2. Konfigurasi
# ------------------------------------------------------------------
# Konfigurasi path, nama model, dan parameter training.
# ------------------------------------------------------------------
# --- Path File ---
FINETUNING_DATASET_JSON_FILE = '/content/fintune_data.json'
MODEL_SAVE_PATH = '/content/custom_finetuned_model.pth'

# --- Model ---
BASE_MODEL_NAME = 'all-MiniLM-L6-v2'

# --- Parameter Training ---
NUM_EPOCHS = 5
BATCH_SIZE = 8
LEARNING_RATE = 2e-5

# Cek ketersediaan GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {device}")

os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)


#@title 3. Persiapan Dataset
# ------------------------------------------------------------------
# Menyiapkan kelas Dataset PyTorch kustom untuk memuat file
# 'fintune_data.json' yang Anda unggah.
# ------------------------------------------------------------------

# Pastikan Anda sudah mengunggah file 'fintune_data.json' ke direktori /content/ di Colab.

# Kelas Dataset kustom
class FineTuningDataset(Dataset):
    def __init__(self, dataset_path):
        try:
            with open(dataset_path, 'r', encoding='utf-8') as f:
                self.data = json.load(f)
        except FileNotFoundError:
            print(f"🛑 Error: File '{dataset_path}' tidak ditemukan. Mohon unggah file tersebut.")
            self.data = []

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return (item['query'], item['positive_passage'], item['negative_passage'])

# Buat instance dataset dan dataloader
train_dataset = FineTuningDataset(FINETUNING_DATASET_JSON_FILE)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)

if len(train_dataset) > 0:
    print(f"✅ Dataset siap digunakan dengan {len(train_dataset)} contoh data.")
else:
    print("⚠️ Dataset kosong. Proses training tidak akan berjalan.")

Menggunakan device: cuda
✅ Dataset siap digunakan dengan 136 contoh data.


In [ ]:
if len(train_dataset) > 0:
    print("\nContoh data dari dataset train:")
    for i in range(min(3, len(train_dataset))):
        query, positive, negative = train_dataset[i]
        print(f"  Contoh {i+1}:")
        print(f"    Query: {query}")
        print(f"    Positive: {positive}")
        print(f"    Negative: {negative}")
else:
    print("\nDataset train kosong, tidak ada data untuk dicetak.")


Contoh data dari dataset train:
  Contoh 1:
    Query: Apa definisi dan peran utama dari Sistem Operasi?
    Positive: Sistem Operasi (Operating System/OS) adalah sebuah program atau perangkat lunak yang mengontrol eksekusi aplikasi. [1]
OS bertindak sebagai antarmuka (interface) antara pengguna (user) dan perangkat keras (hardware) komputer. [1]
OS menyediakan lingkungan di mana pengguna dapat menjalankan program-programnya dengan nyaman dan efisien. [1]
    Negative: Selama eksekusinya, sebuah proses dapat berada dalam salah satu dari beberapa status berikut:
- New (Baru): Proses sedang dibuat.
- Running (Berjalan): Instruksi sedang dieksekusi pada CPU.
- Waiting (Menunggu) / Blocked: Proses sedang menunggu suatu event terjadi (misalnya, penyelesaian I/O atau penerimaan sinyal).
- Ready (Siap): Proses sedang menunggu untuk dialokasikan ke prosesor oleh scheduler.
- Terminated (Selesai): Proses telah selesai dieksekusi.
  Contoh 2:
    Query: Sebutkan dua tujuan utama dari OS.
    Po

In [ ]:
#@title 4. Membangun & Menjalankan Loop Training
# ------------------------------------------------------------------
if len(train_dataset) > 0:
    # 1. Inisialisasi Model, Loss, dan Optimizer
    model = SentenceTransformer(BASE_MODEL_NAME).to(device)
    loss_function = nn.CosineEmbeddingLoss()
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

    print("🚀 Memulai proses training (AI Learning)...")

    # 2. Loop Training
    model.train()

    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        for batch in train_dataloader:
            queries, positive_passages, negative_passages = batch

            # Reset gradien
            optimizer.zero_grad()

            # ---- Mengolah data dan menghitung loss ----
            # Siapkan pasangan positif dan negatif dalam satu batch untuk efisiensi
            # Mengubah tuple menjadi list agar bisa diproses oleh tokenizer
            texts = list(queries) + list(positive_passages) + list(negative_passages)

            # Tokenisasi semua teks sekaligus
            features = model.tokenize(texts)
            # Pindahkan semua tensor hasil tokenisasi ke GPU
            for key in features:
                features[key] = features[key].to(device)

            # Dapatkan semua embedding dengan melewatkan fitur melalui model
            # Ini akan menghasilkan embedding yang memiliki gradien
            embeddings = model(features)['sentence_embedding']

            # Pisahkan kembali embeddingnya sesuai urutan awal
            query_emb = embeddings[0:len(queries)]
            pos_emb = embeddings[len(queries):len(queries)+len(positive_passages)]
            neg_emb = embeddings[len(queries)+len(positive_passages):]

            # Hitung loss untuk pasangan positif (target = 1)
            positive_loss = loss_function(query_emb, pos_emb, torch.ones(len(queries)).to(device))
            # Hitung loss untuk pasangan negatif (target = -1)
            negative_loss = loss_function(query_emb, neg_emb, torch.tensor([-1] * len(queries)).to(device))

            loss = positive_loss + negative_loss

            # --- Perbaiki model ---
            loss.backward()  # Backpropagation: hitung bagaimana cara memperbaiki kesalahan
            optimizer.step() # Terapkan perbaikan (update bobot) pada model

            total_loss += loss.item()

        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{NUM_EPOCHS} - Rata-rata Loss: {avg_loss:.4f}")

    print("✅ Proses training selesai.")

    # Simpan state dictionary dari model yang sudah dilatih
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    print(f"💾 Model yang telah dilatih disimpan di: {MODEL_SAVE_PATH}")
else:
    print("🛑 Training dilewati karena dataset tidak dimuat.")


🚀 Memulai proses training (AI Learning)...
Epoch 1/5 - Rata-rata Loss: 0.7473
Epoch 2/5 - Rata-rata Loss: 0.5912
Epoch 3/5 - Rata-rata Loss: 0.4851
Epoch 4/5 - Rata-rata Loss: 0.4178
Epoch 5/5 - Rata-rata Loss: 0.3760
✅ Proses training selesai.
💾 Model yang telah dilatih disimpan di: /content/custom_finetuned_model.pth


In [32]:
#@title 5. Uji Coba: Membandingkan Model Sebelum dan Sesudah "Belajar"
# ------------------------------------------------------------------
# Menguji kemampuan model dasar vs. model yang kita latih untuk
# menunjukkan dampak nyata dari proses fine-tuning.
# ------------------------------------------------------------------

# Muat materi kuliah untuk pengujian
try:
    # ANDA PERLU MENGUNGGAH FILE INI KE SESI COLAB ANDA
    # Nama file bisa: 'processed_chunks_metadata_base.json', 'processed_chunks_metadata_kecerdasan_buatan.json', dll.
    with open('/content/processed_chunks_metadata_base.json', 'r', encoding='utf-8') as f:
        all_chunks = json.load(f)
    all_passages_for_test = [chunk['chunk_text'] for chunk in all_chunks]
    print(f"✅ Berhasil memuat {len(all_passages_for_test)} passage untuk pengujian.")
except Exception as e:
    all_passages_for_test = []
    print(f"🛑 Gagal memuat passage untuk pengujian. Pastikan file materi sudah diunggah ke /content/: {e}")


if all_passages_for_test and os.path.exists(MODEL_SAVE_PATH):
    # 1. Inisialisasi kedua model
    base_model = SentenceTransformer(BASE_MODEL_NAME).to(device)

    # Inisialisasi ulang arsitektur dan muat bobot yang telah kita latih
    finetuned_model = SentenceTransformer(BASE_MODEL_NAME)
    finetuned_model.load_state_dict(torch.load(MODEL_SAVE_PATH))
    finetuned_model.to(device)

    base_model.eval()
    finetuned_model.eval()

    # 2. Buat embedding untuk semua materi menggunakan KEDUA model
    print("📚 Membuat embedding untuk semua materi...")
    with torch.no_grad():
        corpus_embeddings_base = base_model.encode(all_passages_for_test, convert_to_tensor=True, device=device)
        corpus_embeddings_finetuned = finetuned_model.encode(all_passages_for_test, convert_to_tensor=True, device=device)

    # 3. Lakukan pengujian dengan query yang lebih strategis
    test_queries = [
        "apa kelemahan dari algoritma penjadwalan FCFS?",
        "apa yang dimaksud dengan context switch?",
        "apa itu multiprogramming"
    ]

    print("\n" + "="*50 + "\nHASIL PERBANDINGAN MODEL\n" + "="*50)

    for query in test_queries:
        print(f"\n\n❓ PERTANYAAN: '{query}'")
        print("-" * 40)

        # --- PENGUJIAN MODEL DASAR (SEBELUM BELAJAR) ---
        with torch.no_grad():
            query_embedding_base = base_model.encode(query, convert_to_tensor=True, device=device)
        cos_scores_base = util.cos_sim(query_embedding_base, corpus_embeddings_base)[0]
        top_result_base = torch.topk(cos_scores_base, k=1)

        print("--- 1. Hasil Model Dasar (Belum Belajar) ---")
        for score, idx in zip(top_result_base[0], top_result_base[1]):
            print(f"Skor Relevansi: {score.item():.4f}")
            print(f"Teks Ditemukan:\n{all_passages_for_test[idx]}")

        # --- PENGUJIAN MODEL KUSTOM (SETELAH BELAJAR) ---
        with torch.no_grad():
            query_embedding_finetuned = finetuned_model.encode(query, convert_to_tensor=True, device=device)
        cos_scores_finetuned = util.cos_sim(query_embedding_finetuned, corpus_embeddings_finetuned)[0]
        top_result_finetuned = torch.topk(cos_scores_finetuned, k=1)

        print("\n--- 2. Hasil Model Kustom (Sudah Belajar) ---")
        for score, idx in zip(top_result_finetuned[0], top_result_finetuned[1]):
            print(f"Skor Relevansi: {score.item():.4f}")
            print(f"Teks Ditemukan:\n{all_passages_for_test[idx]}")

else:
    print("🛑 Pengujian dilewati. Pastikan model telah dilatih dan data materi tersedia.")

✅ Berhasil memuat 37 passage untuk pengujian.
📚 Membuat embedding untuk semua materi...

HASIL PERBANDINGAN MODEL


❓ PERTANYAAN: 'apa kelemahan dari algoritma penjadwalan FCFS?'
----------------------------------------
--- 1. Hasil Model Dasar (Belum Belajar) ---
Skor Relevansi: 0.5286
Teks Ditemukan:
- **Konsep:** Proses yang meminta CPU pertama kali akan dilayani pertama kali. [53, 54] Implementasi paling sederhana menggunakan antrian FIFO (First-In, First-Out). Ketika sebuah proses masuk ke ready queue, PCB-nya ditautkan ke akhir antrian. Ketika CPU bebas, ia dialokasikan ke proses di awal antrian.
- **Sifat:** Non-preemptive. Sekali CPU diberikan ke suatu proses, proses tersebut akan menjalankannya hingga selesai atau melakukan permintaan I/O.
- **Kelemahan:** Average waiting time (AWT) seringkali cukup panjang. Efek konvoi (convoy effect) dapat terjadi jika proses dengan burst time panjang datang lebih dulu dan membuat proses-proses pendek di belakangnya menunggu lama. Tidak coco